In [1]:
# GPU: 32*40 in 10.9s = 117/s
# CPU: 32*8 in 31.2s = 8/s

In [2]:
import os
import sys
import numpy as np
import cntk as C
from cntk import load_model, combine
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("CNTK: ", C.__version__)

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Numpy:  1.13.3
CNTK:  2.2


In [3]:
!cat /proc/cpuinfo | grep processor | wc -l

6


In [4]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla K80


In [5]:
BATCH_SIZE = 32
RESNET_FEATURES = 2048
BATCHES_GPU = 40
BATCHES_CPU = 8

In [6]:
def give_fake_data(batches):
    """ Create an array of fake data to run inference on"""
    np.random.seed(0)
    dta = np.random.rand(BATCH_SIZE*batches, 224, 224, 3).astype(np.float32)
    return dta, np.swapaxes(dta, 1, 3)

In [7]:
def yield_mb(X, batchsize):
    """ Function yield (complete) mini_batches of data"""
    for i in range(len(X)//batchsize):
        yield i, X[i*batchsize:(i+1)*batchsize]

In [8]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCHES_GPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(1280, 224, 224, 3) (1280, 3, 224, 224)


In [9]:
#%%bash
#wget https://www.cntk.ai/Models/CNTK_Pretrained/ResNet50_ImageNet_CNTK.model

In [10]:
# Features (penultimate layer)
node_name = "z.x"
model_file = "ResNet50_ImageNet_CNTK.model"

In [11]:
def predict_fn(classifier, data, batchsize):
    """ Return features from classifier """
    out = np.zeros((len(data), RESNET_FEATURES), np.float32)
    for idx, dta in yield_mb(data, batchsize):
        pred = classifier.eval(dta)
        out[idx*batchsize:(idx+1)*batchsize] = pred[0].squeeze()
    return out

## 1. GPU

In [12]:
# Load model
loaded_model  = load_model(model_file)
node_in_graph = loaded_model.find_by_name(node_name)
output_nodes  = combine([node_in_graph.owner])

In [13]:
fake_input_data_cf = np.ascontiguousarray(fake_input_data_cf)
cold_start = predict_fn(output_nodes, fake_input_data_cf, BATCH_SIZE)

In [14]:
%%time
# GPU: 10.9s
features = predict_fn(output_nodes, fake_input_data_cf, BATCH_SIZE)

CPU times: user 7.03 s, sys: 1.6 s, total: 8.62 s
Wall time: 10.9 s


## 2. CPU

In [12]:
# HACK -> have to manually restart notebook and rerun
# Otherwise runs on GPU!!!!!

In [13]:
# Kill all GPUs ...
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [14]:
# Load model
loaded_model  = load_model(model_file)
node_in_graph = loaded_model.find_by_name(node_name)
output_nodes  = combine([node_in_graph.owner])

In [15]:
# Create batches of fake data
fake_input_data_cl, fake_input_data_cf = give_fake_data(BATCHES_CPU)
print(fake_input_data_cl.shape, fake_input_data_cf.shape)

(256, 224, 224, 3) (256, 3, 224, 224)


In [16]:
fake_input_data_cf = np.ascontiguousarray(fake_input_data_cf)
cold_start = predict_fn(output_nodes, fake_input_data_cf, BATCH_SIZE)

In [17]:
%%time
# CPU: 31.2s
features = predict_fn(output_nodes, fake_input_data_cf, BATCH_SIZE)

CPU times: user 2min 46s, sys: 2.22 s, total: 2min 48s
Wall time: 31.2 s
